In [ ]:
import google.generativeai as genai  # Google Gemini API

class LogicalAgent:
    def __init__(self):
        self.facts = set()
        self.rules = {}

    def tell_fact(self, fact):
        """Bir mantıksal fact ekle"""
        self.facts.add(fact)

    def tell_rule(self, rule):
        """Koşul → aksiyon formatında kural ekle"""
        if "->" not in rule:
            raise ValueError("Rule must be in the format 'condition -> action'")
        condition, action = rule.split("->")
        condition = condition.strip()
        action = action.strip()
        self.rules.setdefault(condition, []).append(action)

    def evaluate_for_fact(self, fact):
        actions_list = self.rules.get(fact, [])
        triggered_actions = []

        for actions_str in actions_list:
            actions_str = actions_str.strip()
            if " and " in actions_str:
                # Tüm aksiyonları liste olarak döndür
                actions = [a.strip() for a in actions_str.split(" and ")]
                triggered_actions.extend(actions)
            elif " or " in actions_str:
                # Or varsa, sadece ilk aksiyonu döndür (örnek seçim)
                actions = [a.strip() for a in actions_str.split(" or ")]
                if actions:
                    triggered_actions.append(actions[0])
            else:
                triggered_actions.append(actions_str)

        return triggered_actions


    def evaluate_all(self):
        triggered_actions = []

        for condition, actions_list in self.rules.items():
            condition = condition.strip()

            # Koşul kontrolü (AND / OR / tek fact)
            if " and " in condition:
                facts = [fact.strip() for fact in condition.split(" and ")]
                condition_met = all(fact in self.facts for fact in facts)
            elif " or " in condition:
                facts = [fact.strip() for fact in condition.split(" or ")]
                condition_met = any(fact in self.facts for fact in facts)
            else:
                condition_met = (condition in self.facts)

            if not condition_met:
                continue

            # Koşul sağlandı, her aksiyonu işle
            for actions_str in actions_list:
                actions_str = actions_str.strip()
                if " and " in actions_str:
                    actions = [a.strip() for a in actions_str.split(" and ")]
                    triggered_actions.extend(actions)
                elif " or " in actions_str:
                    actions = [a.strip() for a in actions_str.split(" or ")]
                    if actions:
                        triggered_actions.append(actions[0])
                else:
                    triggered_actions.append(actions_str)

        return triggered_actions




def process_natural_input(input_text):
    input_text = input_text.lower()

    if "üşüyorum" in input_text or "klima" in input_text:
        return "cold_weather"
    elif "karanlık" in input_text or "ışık yok" in input_text:
        return "dark_outside"
    elif "televizyon" in input_text or "tv" in input_text:
        return "user_says_watch_tv"
    elif "geldim" in input_text or "evdeyim" in input_text:
        return "user_arrives_home"
    elif "sabah" in input_text or "güneş doğdu" in input_text:
        return "morning_time"
    elif "perdeyi aç" in input_text:
        return "user_wants_light"

    return call_gemini_api(input_text)


def call_gemini_api(input_text):
    API_KEY = "---------------------------"  # Buraya gerçek API anahtarınızı girin
    genai.configure(api_key=API_KEY)

    model = genai.GenerativeModel('models/gemini-1.5-flash')

    prompt = (
        f"Aşağıdaki doğal dil komutunu analiz et ve yalnızca bir tane kısa mantıksal fact üret. "
        f"Sadece fact'i döndür. Hiçbir açıklama yazma.\n"
        f"Komut: '{input_text}'\n"
        f"Cevap:"
    )

    try:
        response = model.generate_content(prompt)
        content = response.text.strip().lower().replace(" ", "_")
        return content
    except Exception as e:
        print("Gemini API hatası:", e)
        return None


def main():
    agent = LogicalAgent()

   # agent.tell_rule("cold_weather -> turn_on_heater")
    agent.tell_rule("dark_outside -> turn_on_lamp")
    #agent.tell_rule("user_says_watch_tv -> turn_on_tv")
    agent.tell_rule("user_arrives_home -> unlock_door")
   # agent.tell_rule("morning_time -> open_curtains")
    agent.tell_rule("user_wants_light -> open_curtains")


    # Rullerda and or kullanılıyor
    agent.tell_rule("cold_weather -> turn_on_heater and close_windows")
    agent.tell_rule("morning_time -> open curtains and open_windows")
    agent.tell_rule("user_says_watch_tv -> turn_on_tv or dim_lights")

    #Factlerde and or kullanılıyor
    # Gelişmiş kurallar - AND / OR kombinasyonları
    agent.tell_rule("user_arrives_home and dark_outside -> turn_on_entrance_lamp")
    agent.tell_rule("cold_weather and morning_time -> start_heater_and_coffee")
    agent.tell_rule("user_says_watch_tv and dark_outside -> prepare_cinema_mode")
    agent.tell_rule("user_arrives_home and cold_weather -> offer_hot_drink")
    agent.tell_rule("user_wants_light and dark_outside -> maximize_room_lighting")
    agent.tell_rule("user_arrives_home and dark_outside -> turn_on_entrance_lamp")



    inputs = [
        "Üşüyorum",
        "Karanlık oldu",
        "Televizyonu açar mısın?",
        "Geldim",
        "Sabah oldu",
        "Perdeyi aç",
        "Klima çalışmıyor"
    ]

    print("\n Kullanıcı Komutları ve Fact & Aksiyonlar:")
    for user_input in inputs:
        print(f"\n🔹 Girdi: {user_input}")
        fact = process_natural_input(user_input)
        if fact:
            print(f"   Tanımlanan fact: {fact}")
            agent.tell_fact(fact)
            actions = agent.evaluate_for_fact(fact)
            if actions:
                print(f"   Tetiklenen aksiyonlar:")
                for action in actions:
                    print(f"    → {action}")
            else:
                print("   Bu fact için tanımlı aksiyon yok.")
        else:
            print("   Fact çıkarılamadı.")

    #  EK: Birden fazla girdiyle AND/OR kural eşleşmesi:
    combined_inputs = ["üşüyorum", "eve yeni geldim"]
    print("\n Çoklu Girdiye Dayalı Rule Eşleşmesi:")
    print(f"\n🔹 Girdi: Üşüyorum ve Geldim")

    combined_facts = []
    for user_input in combined_inputs:
        fact = process_natural_input(user_input)
        if fact:
            combined_facts.append(fact)

    matched = evaluate_combined_facts(agent, combined_facts)
    if matched:
        for condition, actions in matched:
            print(f"\n Koşul eşleşti: '{condition}'")
            for action in actions:
                print(f"   → {action}")
    else:
        print("  Eşleşen kural yok.")



    print("\n Tüm Fact'lerden Tetiklenen Aksiyonlar:")
    all_actions = agent.evaluate_all()
    if all_actions:
        for action in all_actions:
            print(f"→ {action}")
    else:
        print("Hiçbir kural tetiklenmedi.")



if __name__ == "__main__":
    main()



 Kullanıcı Komutları ve Fact & Aksiyonlar:

🔹 Girdi: Üşüyorum
   Tanımlanan fact: cold_weather
   Tetiklenen aksiyonlar:
    → turn_on_heater
    → close_windows

🔹 Girdi: Karanlık oldu
   Tanımlanan fact: dark_outside
   Tetiklenen aksiyonlar:
    → turn_on_lamp

🔹 Girdi: Televizyonu açar mısın?
   Tanımlanan fact: user_says_watch_tv
   Tetiklenen aksiyonlar:
    → turn_on_tv

🔹 Girdi: Geldim
   Tanımlanan fact: user_arrives_home
   Tetiklenen aksiyonlar:
    → unlock_door

🔹 Girdi: Sabah oldu
   Tanımlanan fact: morning_time
   Tetiklenen aksiyonlar:
    → open curtains
    → open_windows

🔹 Girdi: Perdeyi aç
   Tanımlanan fact: user_wants_light
   Tetiklenen aksiyonlar:
    → open_curtains

🔹 Girdi: Klima çalışmıyor
   Tanımlanan fact: cold_weather
   Tetiklenen aksiyonlar:
    → turn_on_heater
    → close_windows

 Çoklu Girdiye Dayalı Rule Eşleşmesi:

🔹 Girdi: Üşüyorum ve Geldim

 Koşul eşleşti: 'user_arrives_home'
   → unlock_door

 Koşul eşleşti: 'cold_weather'
   → turn_on_hea